# チョコボールのエンゼルの出現確率を推定する

In [ ]:
# ライブラリのインポート
import sys, os
import numpy as np
import pandas as pd
import scipy.stats as stats
import itertools
import math
import pymc3 as pm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# プロットする図を綺麗にする
sns.set()

## データを確認する
データ構成は以下の通り。

| columns | description |
|:--------|:------------|
| best_before | 賞味期限. チョコボールの仕様上,月単位. |
| buyer | 購入者. サンプルデータではA,B,Cの３人が購入している. |
| campaign | 開催中のキャンペーンID. |
| taste | フレーバーID. choco_tastes.csvを参照. |
| weight | 総重量 [g] |
| box_weight | 箱重量 [g] |
| ball_number | チョコボールの個数 |
| silver | 銀のエンゼルの有無(0:無し, 1:有り) |
| gold | 金のエンゼルの有無(0:無し, 1:有り) |

- campaign=1は, 金のエンゼルの出現確率は2倍だが, 銀のエンゼルの出現確率は0%である.
- 正味の重量は(weight - box_weight)で算出. 仕様上の正味重量はchoco_tastes.csvを参照. 


In [ ]:
# データの読み込み
data_raw = pd.read_csv('../data/chocoball_raw.csv')
tastes = pd.read_csv('../data/choco_tastes.csv')

print('data_raw.shape:', data_raw.shape)
print('tastes.shape:', tastes.shape)

In [ ]:
data_raw.head()

In [ ]:
tastes

銀のエンゼルの出現確率を予測する際に、`campaign=1`のデータを使わないようにする。
（データから予め除外しているが、一応確認する。）

In [ ]:
df_data = data_raw[data_raw['campaign']!=1]

print('df_data.shape:', df_data.shape)
df_data.groupby(['silver']).count()[['taste']]

## 仮定
- エンゼルはランダムに入っている
- 時期やフレーバーに依って確率は変化しない


## 最尤推定でパラメータを推定する

### モデル設定
- エンゼルの出現を二項分布でモデル化する
  - エンゼルの出現確率を$\theta$とする
  - チョコボールの購入数（試行数）を$n$とする
$$
f(k|\theta) = \binom{n}{k}\theta^{k}(1-\theta)^{n-k}
$$

### 最尤推定量の計算
- 求めたいパラメータはエンゼルの出現確率である$\theta$
- 最尤推定では、対数尤度$\log{L(\theta|X)}$をパラメータ$\theta$で微分して0となる値を推定値とする

$$
L(\theta|X)=\prod^{N}_{i=1}{f(k|\theta)} = \binom{n}{k}\theta^{k}(1-\theta)^{n-k} \\
\log{L(\theta|X)} = \log{\binom{n}{k}\theta^{k}(1-\theta)^{n-k}} \\
\qquad\qquad\qquad\qquad\qquad = \log{\binom{n}{k}} + \theta\log{{k}} + (1-\theta)\log{{n-k}} \\
$$

微分して0となるパラメータ

$$
\frac{d\log{L(\theta|X)}}{d\theta} = 0 \\
\hat{\theta} = \frac{k}{n}
$$

ということで、長々と数式を展開してきたが、結局は標本平均となる。

In [ ]:
theta_l = np.sum(df_data['silver']) / np.size(df_data['silver'].values)
print('estimated value(MLP):', theta_l)

### 推定値の活用
統計モデリングができたら、その結果を活用して様々なことができる。

#### 推定値をパラメータとした二項分布を確認
二項分布は試行数nを所与として、何回あたりを引くかの分布。

In [ ]:
ks = np.arange(0, 20, 1)
ns = [50, 100, 200]

for n in ns:
    # 確率質量関数を計算
    pmf = stats.binom.pmf(ks, n, p=theta_l)
    # Plot
    plt.plot(ks, pmf, label='n_sample={}'.format(n), marker='.')
plt.legend()
plt.xlabel('k (number of angel)')
plt.ylabel('probability')
plt.savefig('binom_mle.png')

#### 何個買えばエンゼルが5個当たるのか？
- 負の二項分布を利用する（ベルヌーイ試行）
  - $k$ : 成功数（エンゼルの出現数）
  - $x$ : k回成功するまでの失敗回数
  - $\theta$ : 1回の成功確率（エンゼルの出現確率）
$$
f(x|\theta) = \binom{k+x-1}{x}\theta^{k}(1-\theta)^{x}
$$

In [ ]:
k = 5
xs = np.arange(k+0, k+300, 1)

# 確率分布の計算
pmf_nb = stats.nbinom.pmf(xs, k, theta_l)
cdf_nb = pmf_nb.cumsum()

# 累積確率が50%を超える位置を算出
first_over_50 = list(cdf_nb).index(cdf_nb[cdf_nb>0.5].min())

# plot
fig = plt.figure(figsize=(13, 4))
ax = fig.subplots(1, 2)
ax[0].plot(xs, pmf_nb)
ax[0].set_title('Probability Mass Function')
ax[0].set_xlabel('False Count')
ax[0].set_ylabel('Probability Mass')
ax[1].plot(xs, cdf_nb)
ax[1].set_title('Cumulative Probability Mass Function')
ax[1].set_xlabel('False Count')
ax[1].set_ylabel('Cum. Probability')
ax[1].set_ylim([0.0, 1.1])
ax[1].vlines(x=first_over_50, ymin=0, ymax=1.0, color="red", label="50% Over")
print('50% Over point:{}, ({}+{})'.format(first_over_50 + k, first_over_50, k))
plt.savefig('purchase_number_mle.png')

## ベイズ推定でパラメータを推定する

### モデル設定
- ベイズの式を思い出す
    - $p(\theta | x) \propto p(x | \theta)p(\theta)$
    - x : n個のチョコボールを開封して出たエンゼルの数
    - $\theta$ : 確率分布のパラメータ（エンゼルの含有率）
    - 尤度$p(x | \theta)$と事前分布$p(\theta)$を設定する必要がある
- 尤度関数
    - 最尤推定と同様に、二項分布を利用
    - $p(x | \theta) = \binom{n}{x}\theta^x(1-\theta)^{N-x}$
- 事前分布
    - ベータ分布
    - $p(\theta) = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}\theta^{\alpha-1}(1-\theta)^{\beta-1}$


#### ベータ分布の形状を見てみる

In [ ]:
params = [0.1, 1, 2, 10] # alpha, betaの例

x = np.linspace(0, 1, 100) # x軸の設定

fig = plt.figure(figsize=(13, 10))
ax = fig.subplots(len(params), len(params), sharex=True, sharey=True)
cnt=0
for i in range(len(params)):
    for j in range(len(params)):
        # パラメータalphaとbetaを設定
        a = params[i]
        b = params[j]
        # ベータ分布の確率密度を計算
        y = stats.beta(a, b).pdf(x)
        # plot
        ax[i, j].plot(x, y)
        ax[i, j].plot(0, 0, label="$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(a, b), alpha=0)
        ax[i, j].legend()
        if i == (len(params)-1):
            ax[i,j].set_xlabel('$\\theta$')
        if j == 0:
            ax[i,j].set_ylabel('$p(\\theta)$')
plt.savefig('beta_dist_var.png')

### 解析的な計算方法(共役事前分布)

ベイズの定理を再度思い出す。
事後分布は尤度関数と事前分布の積に比例するという式である。
$$
p(\theta | y) \propto p(y | \theta)p(\theta)
$$
尤度関数には、二項分布で事前分布はベータ分布と定義したので、
ベイズの定理は以下のような式になる。
$$
p(\theta | y) \propto \frac{N!}{y!(N-y)!}\theta^y(1-\theta)^{N-y}\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}\theta^{\alpha-1}(1-\theta)^{\beta-1}
$$

$\theta$に関係しない部分は比例定数として押し込めてしまうことで、
以下の式が得られる。

$$
p(\theta | y)　\propto \theta^{\alpha-1+y}(1-\theta)^{\beta-1+N-y}
$$

この式はベータ分布に一致する。
$$
p(\theta | y)　= Beta(\alpha_{prior}+y, \beta_{prior}+N-y)
$$

つまり、今回のモデル定義においては、解析的に事後分布を導くことができた。  
このように尤度関数との積が同じ関数になる事前分布を「共役事前分布」と呼ぶ。
共役事前分布をモデルに利用すれば解析的に解を求めることができるが、
もっと複雑なモデルを使う場合には、一般的に解析解が得られない。

### 数値的な計算方法(MCMC)

#### 計算の実行
複雑なモデルや共役でない事前分布を使う場合、計算が困難か解析的には計算が不可能な場合がある。  
このような場合にも事後分布を計算するアルゴリズムとして、
マルコフチェーンモンテカルロ（MCMC）と呼ばれるアルゴリズムがある。  
詳細は省略するが、ざっくりとしたイメージでは、形状がわからない確率分布（事後分布）の大きさに比例してデータをサンプルするアルゴリズムである。

In [ ]:
d_angel = df_data['silver'].values
n_sample = 1000

with pm.Model() as model_single:
    # 事前分布
    theta = pm.Beta('theta', alpha=1, beta=1)
    #theta = pm.Uniform('theta', lower=0, upper=1)
    # 尤度
    y = pm.Binomial('y', n=len(d_angel), p=theta, observed=sum(d_angel))
    #y = pm.Bernoulli('obs', p=theta, observed=d_angel) # ベルヌーイ分布
    # sample
    trace_single = pm.sample(n_sample, chains=4)

pm.traceplot(trace_single)

In [ ]:
ret = pm.model_to_graphviz(model_single)
ret.render(filename='model_single_01', format='png')
ret

#### 結果の解釈

推定対象である、二項分布のパラメータ$\theta$(エンゼルの含有率)の事後分布を確認する。
なお、以下の図はchain数（MCMCサンプル系列の数）分の結果が同時に表示されている。
- 左図:$\theta$の事後分布
- 右図:$\theta$のサンプル系列。ランダムにサンプルされていることが望ましい。

In [ ]:
fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1,2)
ax = ax[np.newaxis, :]

pm.traceplot(trace_single, ax=ax)
ax[0,0].vlines(x=theta_l, ymin=0, ymax=30, color="red", label="MLE")
ax[0,1].hlines(y=theta_l, xmin=0, xmax=n_sample, color="red", label="MLE")

plt.savefig('trace_plot_angel_rate.png')

chainを全て統合して、事後分布を推定。
- 信用区間(HPD)をalpha_levelで指定

In [ ]:
pm.plot_posterior(trace_single, kde_plot=True, alpha_level=0.05)
plt.savefig('posterior_angel_rate.png')

#### 何個買えばエンゼルが5個当たるのか？
- 最尤推定の場合と同様に負の二項分布を利用して推定する

In [ ]:
theta_tr = trace_single['theta']
alpha_level = 0.05
k = 5
xs = np.arange(k+0, k+300, 1)
pmf_nb_ex = stats.nbinom.pmf(xs, k, theta_tr.mean())
pmf_nb_lb = stats.nbinom.pmf(xs, k, np.percentile(theta_tr, (alpha_level*50.0)))
pmf_nb_ub = stats.nbinom.pmf(xs, k, np.percentile(theta_tr, (100.0-alpha_level*50.0)))
cdf_nb_ex = pmf_nb_ex.cumsum()
cdf_nb_lb = pmf_nb_lb.cumsum()
cdf_nb_ub = pmf_nb_ub.cumsum()

ex_p = 0.5
first_over_ex = list(cdf_nb_ex).index(cdf_nb_ex[cdf_nb_ex>=ex_p].min())
first_over_lb = list(cdf_nb_lb).index(cdf_nb_lb[cdf_nb_lb>=ex_p].min())
first_over_ub = list(cdf_nb_ub).index(cdf_nb_ub[cdf_nb_ub>=ex_p].min())

fig = plt.figure(figsize=(13, 4))
ax = fig.subplots(1, 2)

ax[0].plot(xs, pmf_nb_ex)
ax[0].set_title('Probability Mass Function')
ax[0].set_xlabel('False Count')
ax[0].set_ylabel('Probability Mass')
ax[1].plot(xs, cdf_nb_ex)
ax[1].fill_between(xs, cdf_nb_lb, cdf_nb_ub, facecolor='y',alpha=0.5)
ax[1].set_title('Cumulative Probability Mass Function')
ax[1].set_xlabel('False Count')
ax[1].set_ylabel('Cum. Probability')
ax[1].set_ylim([0.0, 1.1])
ax[1].vlines(x=first_over_ex, ymin=0, ymax=1.0, color="red", label="{}% Over (ex)".format(ex_p))
ax[1].vlines(x=first_over_lb, ymin=0, ymax=1.0, color="green", label="{}% Over (lb)".format(ex_p))
ax[1].vlines(x=first_over_ub, ymin=0, ymax=1.0, color="blue", label="{}% Over (ub)".format(ex_p))
print('{}% Over point:{} ~ {} ~ {} (alpha_level={})'.format(ex_p*100, first_over_ub, first_over_ex, first_over_lb, alpha_level))
plt.savefig('purchase_number_bayes.png')

## 購入者毎の差（運の良さ）の有無を確認

- チョコボールの購入者（３人）によって運の良さが違う気がする
- 運の良さというものがあるのかを確認
- この例題を通して、グループ間の比較と階層ベイズモデルを体験

### 最尤推定量を確認

In [ ]:
buyer = df_data['buyer'].values
buyer_idx = pd.Categorical(df_data['buyer']).codes
buyer_cat = pd.Categorical(df_data['buyer']).categories
df_data['buyer_idx'] = buyer_idx
lst_buyer = list(set(buyer_idx))
print(buyer_cat)
print(set(buyer_idx))

In [ ]:
total_counts = df_data.groupby(['buyer_idx']).count()['silver'].values
angel_counts = df_data.query('silver > 0').groupby(['buyer_idx']).count()['silver'].values

print('total_count : {}'.format(total_counts))
print('angel_count : {}'.format(angel_counts))

In [ ]:
theta_mle = angel_counts/total_counts
print(theta_mle)

fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1, 1)
cs = ['#FF4500', '#0000FF', '#00F1A1']
for idx in np.arange(0, len(theta_mle)):
    ax.vlines(theta_mle[idx], 0, 1, colors=cs[idx%len(cs)], label=buyer_cat[idx])
ax.set_xlim((0.0, 0.1))
ax.set_xlabel('$\\theta_{MLE}$')
ax.legend()
plt.savefig('buyer_effect_mle.png')

### グループ間の比較
- 購入者毎にエンゼル出現確率を推定

In [ ]:
with pm.Model() as model_iso:
    # thetaは購入者数分サンプル
    theta = pm.Beta('theta', alpha=1, beta=1, shape=len(set(buyer_idx)))
    
    angel = pm.Binomial('angel', n=total_counts[lst_buyer], p=theta[lst_buyer], observed=angel_counts[lst_buyer])
    
    trace_iso = pm.sample(5000, chains=1, random_seed=100)
pm.traceplot(trace_iso)

In [ ]:
ret = pm.model_to_graphviz(model_iso)
ret.render(filename='model_multi_01', format='png')
ret

#### パラメータの事後分布を確認

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.subplots(1, 1)

cs = ['#0101DF', '#FF8000', '#04B404']
for i in np.arange(len(buyer_cat)):
    sns.distplot(trace_iso['theta'][:,i], label=buyer_cat[i], ax=ax)
    ax.vlines(x=theta_mle[i], ymin=0, ymax=25, color=cs[i], label="MLE_{}".format(buyer_cat[i]))
ax.legend()
ax.set_xlabel('angel rate')
ax.set_ylabel('frequent')
plt.savefig('buyer_effect_iso.png')

#### 差の分布を確認

In [ ]:
# 差の分布を確認
n_c = len(list(itertools.combinations(np.arange(0, len(lst_buyer)), 2)))
n_col = 3
n_row = math.ceil(n_c/n_col)
fig = plt.figure(figsize=(12, 3*n_row))
ax = fig.subplots(n_row, n_col)
if n_row == 1:
    ax = ax[np.newaxis, :]

cnt=0
for (i,j) in itertools.combinations(np.arange(0, len(lst_buyer)), 2):
    theta_diff = trace_iso['theta'][:, i] - trace_iso['theta'][:, j]
    pm.plot_posterior(theta_diff, ref_val=0, ax=ax[int(cnt/n_col), int(cnt%n_col)])
    ax[int(cnt/n_col), int(cnt%n_col)].set_title('{}-{}'.format(buyer_cat[i], buyer_cat[j]))
    cnt+=1
plt.savefig('buyer_effect_diff_isomodel.png')

### 運の要素をパラメータに追加
- 真の出現率は決まっているはず(真の確率を$p$とする)
- 購入者毎の運の要素が入ってくる（不正行為かも、独自の購入戦略があるのかも）可能性がある
  - 購入者毎の特性を個人差$u_i$とする
- エンゼルの出現は確率$\theta_i$の二項分布
- 確率$\theta_i$は以下のロジットリンク関数で線形モデルを仮定
$$
logit(\theta_i) = p + u_i
$$

- $p$、$u_i$は正規分布を事前分布とする

In [ ]:
with pm.Model() as comparing_buyer_m1:
    su = pm.HalfNormal('su', sd=20)
    p = pm.Normal('p', mu=0, sd=20)
    u = pm.Normal('u', mu=0, sd=su, shape=len(set(buyer_idx)))
    
    theta = pm.Deterministic('theta', pm.math.sigmoid(p+u[lst_buyer]))
    angel = pm.Binomial('angel', 
                        n=total_counts[lst_buyer], 
                        p=theta, observed=angel_counts[lst_buyer])
    
    trace_h1 = pm.sample(3000, chains=3, random_seed=100)
pm.traceplot(trace_h1)

In [ ]:
ret = pm.model_to_graphviz(comparing_buyer_m1)
ret.render(filename='model_multi_02', format='png')
ret

In [ ]:
pm.plot_posterior(trace_h1, varnames=['p', 'u'])

#### 購入者毎の個人差（運の要素）の事後分布

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.subplots(1, 1)

for i in np.arange(len(buyer_cat)):
    sns.distplot(trace_h1['u'][:,i], label=buyer_cat[i], ax=ax)
ax.legend()
ax.set_xlabel('angel rate')
ax.set_ylabel('frequent')
plt.savefig('buyer_effect_h1model.png')

In [ ]:
# 差の分布を確認
n_c = len(list(itertools.combinations(np.arange(0, 3), 2)))
n_col = 3
n_row = math.ceil(n_c/n_col)
fig = plt.figure(figsize=(12, 3*n_row))
ax = fig.subplots(n_row, n_col)
if n_row == 1:
    ax = ax[np.newaxis, :]

cnt=0
for (i,j) in itertools.combinations(np.arange(0, 3), 2):
    u_diff = trace_h1['u'][:, i] - trace_h1['u'][:, j]
    pm.plot_posterior(u_diff, ref_val=0, ax=ax[int(cnt/n_col), int(cnt%n_col)])
    ax[int(cnt/n_col), int(cnt%n_col)].set_title('{}-{}'.format(buyer_cat[i], buyer_cat[j]))
    cnt+=1
plt.savefig('buyer_effect_diff_h1model.png')

#### 全体のエンゼルの出現確率の推定
- モデルに寄れば、エンゼルの出現確率$\theta_i$は以下のロジットリンク関数と線形予測子で表現していた
$$
logit(\theta_i) = p + u_i
$$
- そのため、確率に変換するために、ロジスティック・シグモイド関数(下記)に通す必要がある
$$
\sigma(x) = \frac{1}{1+\exp(-x)}
$$

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [ ]:
(post_min, post_max) = pm.hpd(trace_h1['p'], alpha=0.1)
post_mean = pm.summary(trace_h1).loc['p', 'mean']
print('{} < {} < {}'.format(sigmoid(post_min), sigmoid(post_mean), sigmoid(post_max)))